In [3]:
#Conducting Monte Carlo Simulations to Generate and Analyze Single-Case Graphs

#Import packages
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

#This function creates a time series with n points, an autocorrelation of a,
#and a constant of ct

def create_time_series(n, a, ct):

    #Create an empty vector to hold values
    time_series = np.empty((0,))

    #Compute first point (no autocorrelation possible)
    point1 = np.random.normal(size = 1)

    #Add point1 to time series
    time_series = np.hstack((time_series, point1))

    #Repeat the process below for all subsequent points
    for i in range(1, n):

        #Compute autocorrelated point error term
        point = a*time_series[i-1]+np.random.normal(size = 1)

        #Add autcorrelated point to time series
        time_series = np.hstack((time_series, point))

    #Add constant to all points
    time_series = time_series + ct

    #Return the time series
    return(time_series)



In [4]:
#This function creates data for an alternating treatment graph with an
#autocorrelation of a, a trend of tr (in degrees), a constant of ct, a minimum
#of nb_points in each condition, a standardized mean difference of smd,
#and level of r in the control condition. Alternation is semi-random

def create_ME_data(a, trControl, trAttention, trTangibles, trEscape, trAlone, ct, nb_points, smdAttention, smdTangibles, smdEscape, smdAlone, r, alternation):

  if alternation == 'semi-random':

        #Empty vector for labels
        labels = np.empty((0,))

        #Repeat process for each pair of points
        for i in range(nb_points):

            #Randomly select the order of 5 conditions
            new_var = 5
            conditions = np.random.choice(['Control', 'Attention', 'Tangibles', 'Escape', 'Alone'], 5, replace = False)

            #Add conditions to labels
            labels = np.hstack((labels, conditions))

        #Run code until minimum number of points is reached for all phases
        while (np.sum(labels == 'Control') < nb_points) | (np.sum(labels == 'Attention') < nb_points)| (np.sum(labels == 'Tangibles') < nb_points)| (np.sum(labels == 'Escape') < nb_points)| (np.sum(labels == 'Alone') < nb_points):

            #Randomly select one condition
            condition = np.random.choice(['Control', 'Test', 'Tangibles', 'Escape', 'Alone'], 1)

            #Add condition to labels
            labels = np.hstack((labels, condition))

  #Create times series
  time_series = create_time_series(len(labels), a, ct)
  all_values = np.copy(time_series)

  #Indices for Phase A (Control)
  idxControl = np.where(labels == 'Control')[0]

  #Indices for Attention
  idxAttention = np.where(labels == 'Attention')[0]

  #Indices for Tangibles
  idxTangibles = np.where(labels == 'Tangibles')[0]

  #Indices for Escape
  idxEscape = np.where(labels == 'Escape')[0]

  #Indices for Alone
  idxAlone = np.where(labels == 'Alone')[0]

  #Add smd to values of test conditions
  all_values[idxAttention] += smdAttention
  all_values[idxTangibles] += smdTangibles
  all_values[idxEscape] += smdEscape
  all_values[idxAlone] += smdAlone

  #Identify middle point around which to pivot trend
  middle_point = np.median(range(len(all_values)))

  #Apply trend to all points separately
  #Control
  for j in range(len(idxControl)):
    distance = idxControl[j] - middle_point
    all_values[idxControl[j]] += distance * math.tan(trControl * math.pi / 180)

  #Phase Attention
  for j in range(len(idxAttention)):
        distance = idxAttention[j] - middle_point
        all_values[idxAttention[j]] += distance * math.tan(trAttention * math.pi / 180)

  #Phase Tangibles
  for j in range(len(idxTangibles)):
        distance = idxTangibles[j] - middle_point
        all_values[idxTangibles[j]] += distance * math.tan(trTangibles * math.pi / 180)

  #Phase Escape
  for j in range(len(idxEscape)):
        distance = idxEscape[j] - middle_point
        all_values[idxEscape[j]] += distance * math.tan(trEscape * math.pi / 180)

  #Phase Alone
  for j in range(len(idxAlone)):
        distance = idxAlone[j] - middle_point
        all_values[idxAlone[j]] += distance * math.tan(trAlone * math.pi / 180)

  #Set max and min values, and normalize
  min_value = np.min(all_values)
  max_value = np.max(all_values)
  all_values = (all_values - min_value) / (max_value - min_value) * 10

  #Recalculate smd
  mean_control = np.mean(all_values[idxControl])
  mean_attention = np.mean(all_values[idxAttention])
  mean_tangibles = np.mean(all_values[idxTangibles])
  mean_escape = np.mean(all_values[idxEscape])
  mean_alone = np.mean(all_values[idxAlone])

  current_smd_attention = mean_attention - mean_control
  current_smd_tangibles = mean_tangibles - mean_control
  current_smd_escape = mean_escape - mean_control
  current_smd_alone = mean_alone - mean_control

  #Adjust values to maintain smd
  #Attention
  if current_smd_attention != smdAttention:
      adjustment = smdAttention - current_smd_attention
      all_values[idxAttention] += adjustment / 2
      all_values[idxControl] -= adjustment / 2

  #Tangibles
  if current_smd_tangibles != smdTangibles:
      adjustment = smdTangibles - current_smd_tangibles
      all_values[idxTangibles] += adjustment / 2
      all_values[idxControl] -= adjustment / 2

  #Escape
  if current_smd_escape != smdEscape:
      adjustment = smdEscape - current_smd_escape
      all_values[idxEscape] += adjustment / 2
      all_values[idxControl] -= adjustment / 2

  #Alone
  if current_smd_alone != smdAlone:
      adjustment = smdAlone - current_smd_alone
      all_values[idxAlone] += adjustment / 2
      all_values[idxControl] -= adjustment / 2

  #Set max and min values, and normalize (to ensure positive values)
  min_value = np.min(all_values)
  max_value = np.max(all_values)
  all_values = (all_values - min_value) / (max_value - min_value) * 10

  #add r after normalization to increase level
  all_values[idxControl] += r
  all_values[idxAttention] += r
  all_values[idxTangibles] += r
  all_values[idxEscape] += r
  all_values[idxAlone] += r

  #Combine labels and values in same list
  ME_data = [labels, all_values]

  #Return ME data
  return(ME_data)



In [5]:
#Function to produce ME graph

def ATgraph(ME_data, trControl, trAttention, trTangibles, trEscape, trAlone):

    #Identify indices for all conditions
    Control, = np.where(ME_data[0] == 'Control')
    Attention, = np.where(ME_data[0] == 'Attention')
    Tangibles, = np.where(ME_data[0] == 'Tangibles')
    Escape, = np.where(ME_data[0] == 'Escape')
    Alone, = np.where(ME_data[0] == 'Alone')

    #Extract values for each condition
    valuesControl = ME_data[1][Control]
    valuesTest = ME_data[1][Attention]
    valuesTestB = ME_data[1][Tangibles]
    valuesTestC = ME_data[1][Escape]
    valuesTestD = ME_data[1][Alone]

    #Calculate mean and sd of control
    meanControl = np.mean(valuesControl)
    stdControl = np.std(valuesControl)

    #Initialize figure
    fig = plt.figure()
    ax = fig.add_subplot(111)

    #Plot data
    plt.plot(Control+1, valuesControl, 'ks-', label = 'Control')
    plt.plot(Attention+1, valuesTest, 'ko-', label = 'Attention')
    plt.plot(Tangibles+1, valuesTestB, 'go-', label = 'Tangibles')
    plt.plot(Escape+1, valuesTestC, 'ro-', label = 'Escape')
    plt.plot(Alone+1, valuesTestD, 'bo-', label = 'Alone')

    #Add axes titles
    plt.xlabel('Session')
    plt.ylabel('Behavior')

    #Add legend to graph
    ax.legend(loc='upper right', frameon=False)

    #Adjust height of graph
    plt.ylim(0, np.max(ME_data[1]*1.5))

    #Remove right and top borders
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    #Plot UCL and LCL
    if meanControl == 0:
        plt.axhline(y=0, color='b', linestyle='--', label='UCL/LCL at 0')
    else:
        plt.axhline(y=meanControl + stdControl, color='b', linestyle='--', label='UCL')
        plt.axhline(y=meanControl - stdControl, color='b', linestyle='--', label='LCL')


In [6]:
# Define ranges for each parameter
nb_points_list = [6, 10]
a_list = [0, 0.2]
tr_list_Control = [0] #determine number of 0s, angle of trend from effect size data
tr_list_Attention = [0]
tr_list_Tangibles = [0]
tr_list_Escape = [0]
tr_list_Alone = [0]
smd_list_Attention = [0, 3]
smd_list_Tangibles = [0, 3]
smd_list_Escape = [0, 3]
smd_list_Alone = [0, 3]
r_list = [0]
ct = 10


# Function to create data and graph
def perform_simulations():

  #Empty list to contain all the input data
  all_ME_data = []

  #Empty vector to contain the true values
  true_values = np.empty((0,))

  #Empty vector to contain trend values for each data series
  trend_values = np.empty((0, 5))

  #pp
  pp = PdfPages('ME5graphTest.pdf')

  # Loop over each combination of parameters
  for nb in nb_points_list:
       for a in a_list:
          for trControl in tr_list_Control:
              for trAttention in tr_list_Attention:
                  for trTangibles in tr_list_Tangibles:
                      for trEscape in tr_list_Escape:
                          for trAlone in tr_list_Alone:
                              for smdAttention in smd_list_Attention:
                                  for smdTangibles in smd_list_Tangibles:
                                      for smdEscape in smd_list_Escape:
                                          for smdAlone in smd_list_Alone:
                                              for r in r_list:

                                                  # Generate ME data for the set of parameters
                                                  ME_data = create_ME_data(a=a, trControl=trControl, trAttention=trAttention, trTangibles=trTangibles, trEscape=trEscape, trAlone=trAlone, ct=ct, nb_points=nb, smdAttention=smdAttention, smdTangibles=smdTangibles, smdEscape=smdEscape, smdAlone=smdAlone, r=r, alternation='semi-random')

                                                  # Plot graph
                                                  plt.figure()
                                                  ATgraph(ME_data, trControl, trAttention, trTangibles, trEscape, trAlone)

                                                  # Title to distinguish different graphs
                                                  plt.title(f'nb_points={nb}, a={a}, trControl={trControl}, trAttnention={trAttention}, trTangibles={trTangibles},trEscape={trEscape},trControl={trControl}, smdAttention={smdAttention}, smdTangibles={smdTangibles}, smdEscape={smdEscape}, smdAlone={smdAlone}, r={r}', fontsize=4)

                                                  #plt.show()
                                                  pp.savefig()
                                                  plt.close()

                                                  #Add data series to list
                                                  all_ME_data.append(ME_data)

                                                  #Determine true values
                                                  smd_values = [smdAttention, smdTangibles, smdEscape, smdAlone]
                                                  count_greater_than_zero = sum(1 for smd in smd_values if smd > 0)
                                                  true_value = 2 if count_greater_than_zero >= 2 else (1 if count_greater_than_zero == 1 else 0)

                                                  #Add trend value to vector
                                                  trend_values = np.vstack((trend_values, np.array([[trControl, trAttention, trTangibles, trEscape, trAlone]])))

                                                  #Add true value to vector
                                                  true_values = np.hstack((true_values, true_value))

  pp.close()

  return all_ME_data, true_values, trend_values

#Call
all_ME_data, true_values, trend_values = perform_simulations()


<ipython-input-5-82444e1222f7>:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()
<ipython-input-6-d4d4a0592169>:50: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [7]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import tan, radians
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.cluster import KMeans

lm = LinearRegression()
standard_scaler = preprocessing.StandardScaler()

# List of condition names
conditions_names = ['Control', 'Attention', 'Tangibles', 'Escape', 'Alone']

In [12]:
def standardize_data(x):

    features = []
    conditions = []  #Initialize conditions list

    #identify indicies
    indexControl = np.where(x[0]== 'Control')
    indexAttention = np.where(x[0]== 'Attention')
    indexTangibles = np.where(x[0]== 'Tangibles')
    indexEscape = np.where(x[0]== 'Escape')
    indexAlone = np.where(x[0]== 'Alone')

    #extract data
    Control = (x[1][indexControl].astype(float)).flatten()
    Attention = (x[1][indexAttention].astype(float)).flatten()
    Tangibles = (x[1][indexTangibles].astype(float)).flatten()
    Escape = (x[1][indexEscape].astype(float)).flatten()
    Alone = (x[1][indexAlone].astype(float)).flatten()

    conditions.extend([Control, Attention, Tangibles, Escape, Alone]) # Add conditions to the list

    #mean and SD
    overallMean = np.mean(np.hstack((Control, Attention, Tangibles, Escape, Alone)))
    overallSd = np.std(np.hstack((Control, Attention, Tangibles, Escape, Alone)))

    #standardize
    Control  = (Control-overallMean)/overallSd
    Control[np.isnan(Control)] = 0

    Attention = (Attention-overallMean)/overallSd
    Attention[np.isnan(Attention)] = 0

    Tangibles = (Tangibles-overallMean)/overallSd
    Tangibles[np.isnan(Tangibles)] = 0

    Escape = (Escape-overallMean)/overallSd
    Escape[np.isnan(Escape)] = 0

    Alone = (Alone-overallMean)/overallSd
    Alone[np.isnan(Alone)] = 0

    pointsControl = len(Control)
    pointsTotal = pointsControl + len(Attention) + len(Tangibles) + len(Escape) + len(Alone)


    #Collect basic statistics: mean and standard deviation for each condition
    features.extend([np.mean(Control), np.mean(Attention), np.mean(Tangibles), np.mean(Escape), np.mean(Alone)])
    features.extend([np.std(Control), np.std(Attention), np.std(Tangibles), np.std(Escape), np.std(Alone)])


    #Fit linear models for each condition to collect slope and intercept (linear trend)
    for condition in conditions:
        if len(condition) > 0:  # Avoid fitting if condition data is empty
            lm.fit(np.array(range(len(condition))).reshape(-1, 1), condition.reshape(-1, 1))
            features.append(float(lm.intercept_))  # Append intercept of regression line
            features.append(float(lm.coef_))       # Append slope of regression line
        else:
            # If condition is empty, append zeros for both intercept and slope
            features.append(0)
            features.append(0)

    #Calculate the Interquartile Range (IQR) for each condition
    for condition in conditions:
        if len(condition) > 0:  #Avoid calculation if condition data is empty
            q75, q25 = np.percentile(condition, [75, 25])
            iqr = q75 - q25
            features.append(iqr)  #Append IQR for the condition
        else:
            features.append(0)   #If condition is empty, append 0 for IQR

    return np.array(features).flatten()

In [13]:
#Call
all_ME_data, true_values, trend_values = perform_simulations()

#Collect all features using the standardize_data function
all_features = []
expected_feature_length = 39

for i, ME_data in enumerate(all_ME_data):
    try:
        #basic features from standardize_data
        features = standardize_data(ME_data)

        #Extract raw conditions from data
        Control = (ME_data[1][np.where(ME_data[0] == 'Control')].astype(float)).flatten()
        Attention = (ME_data[1][np.where(ME_data[0] == 'Attention')].astype(float)).flatten()
        Tangibles = (ME_data[1][np.where(ME_data[0] == 'Tangibles')].astype(float)).flatten()
        Escape = (ME_data[1][np.where(ME_data[0] == 'Escape')].astype(float)).flatten()
        Alone = (ME_data[1][np.where(ME_data[0] == 'Alone')].astype(float)).flatten()

        #Stability (Standard Deviation / Mean) for each condition
        stability_conditions = [Control, Attention, Tangibles, Escape, Alone]
        for condition in stability_conditions:
            stability = np.std(condition) / np.mean(condition) if np.mean(condition) != 0 else np.inf
            features = np.append(features, stability)

        #Percentage of Non-Overlapping Data (PND) with Control
        def calculate_pnd(control_data, comparison_data):
            if len(control_data) == 0 or len(comparison_data) == 0:
                return 0  # If either dataset is empty, return 0 for PND
            threshold = np.max(control_data)
            non_overlapping_points = [point for point in comparison_data if point > threshold]
            pnd = len(non_overlapping_points) / len(comparison_data)
            return pnd

        #Calculate PND for each condition compared to Control
        features = np.append(features, calculate_pnd(Control, Attention))
        features = np.append(features, calculate_pnd(Control, Tangibles))
        features = np.append(features, calculate_pnd(Control, Escape))
        features = np.append(features, calculate_pnd(Control, Alone))

        #Highest Response Condition
        condition_means = [np.mean(Control), np.mean(Attention), np.mean(Tangibles), np.mean(Escape), np.mean(Alone)]
        highest_response_condition = np.argmax(condition_means)  # Store as an integer representing index
        features = np.append(features, highest_response_condition)

        #Condition Clustering (KMeans)
        kmeans_data = np.array([condition_means]).reshape(-1, 1)  # Corrected reshape
        kmeans = KMeans(n_clusters=2, random_state=0).fit(kmeans_data)
        cluster_labels = kmeans.labels_

        #Identify if each condition falls in the same cluster as Control
        control_cluster = cluster_labels[0]
        for label in cluster_labels[1:]:
            features = np.append(features, int(label == control_cluster))

        #Debugging: Check the length of features
        if len(features) != expected_feature_length:
            print(f"Inconsistent feature length at index {i}: {len(features)} (expected {expected_feature_length})")
            # Add padding if features are shorter, to maintain consistent length
            if len(features) < expected_feature_length:
                features = np.append(features, [0] * (expected_feature_length - len(features)))

        #Add the complete feature set to all_features
        all_features.append(features)

    except Exception as e:
        print(f"Error processing data at index {i}: {e}")

#Ensure that all_features has at least one element before proceeding
if not all_features:
    raise ValueError("No features were generated. Please check the input data and feature extraction logic.")

#Convert features to a DataFrame
columns = [
    'Mean Control', 'Mean Attention', 'Mean Tangibles', 'Mean Escape', 'Mean Alone',
    'Std Control', 'Std Attention', 'Std Tangibles', 'Std Escape', 'Std Alone',
    'Intercept Control', 'Slope Control',
    'Intercept Attention', 'Slope Attention',
    'Intercept Tangibles', 'Slope Tangibles',
    'Intercept Escape', 'Slope Escape',
    'Intercept Alone', 'Slope Alone',
    'IQR Control', 'IQR Attention', 'IQR Tangibles', 'IQR Escape', 'IQR Alone',
    'Stability Control', 'Stability Attention', 'Stability Tangibles', 'Stability Escape', 'Stability Alone',
    'PND Control-Attention', 'PND Control-Tangibles', 'PND Control-Escape', 'PND Control-Alone',
    'Highest Response Condition',
    'Cluster Similarity Attention', 'Cluster Similarity Tangibles', 'Cluster Similarity Escape', 'Cluster Similarity Alone'
]

#Correct length of features before creating DataFrame
all_features = np.array(all_features)
assert all_features.shape[1] == len(columns), "Mismatch between number of features and column names."

df_features = pd.DataFrame(all_features, columns=columns)

#Save DataFrame to CSV
df_features.to_csv('std_features_5condition.csv', index=False)

print("Features saved to std_features_5condition.csv")

<ipython-input-5-82444e1222f7>:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()
<ipython-input-6-d4d4a0592169>:50: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()
<ipython-input-12-668b296618b7>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  features.

Features saved to std_features_5condition.csv


<ipython-input-12-668b296618b7>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  features.append(float(lm.intercept_))  # Append intercept of regression line
<ipython-input-12-668b296618b7>:56: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  features.append(float(lm.coef_))       # Append slope of regression line
<ipython-input-12-668b296618b7>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  features.append(float(lm.intercept_))  # Append intercept of regression l

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>